<a href="https://colab.research.google.com/github/shrisrivas/google-co_lab_prog/blob/main/cuda_winequality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## classification

In [1]:
!nvidia-smi

Thu Sep 15 09:12:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 26.3 MB/s 
     |████████████████████████████████| 209 kB 59.7 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 112 kB 65.5 MB/s 
     |████████████████████████████████| 49 kB 6.7 MB/s 
     |████████████████████████████████| 147 kB 72.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=28f092c58901a1b65a4a0bc88df2de6df4a0d35a421ba3fbd819fa70bf7eb766
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/shrisrivas/google_colab_datasets/main/winequality-red.csv',sep=';')

In [6]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [7]:
x = df.drop(columns='quality')
y = df.quality

In [8]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [9]:
train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )

In [10]:
np.asarray(train_x)

array([[6.00e+00, 5.00e-01, 0.00e+00, ..., 3.36e+00, 4.50e-01, 9.50e+00],
       [7.40e+00, 5.80e-01, 0.00e+00, ..., 3.45e+00, 5.80e-01, 1.13e+01],
       [7.10e+00, 4.70e-01, 0.00e+00, ..., 3.40e+00, 5.80e-01, 1.09e+01],
       ...,
       [7.10e+00, 5.90e-01, 1.00e-02, ..., 3.42e+00, 5.80e-01, 1.07e+01],
       [8.20e+00, 3.40e-01, 3.80e-01, ..., 3.30e+00, 4.70e-01, 9.00e+00],
       [7.80e+00, 5.50e-01, 3.50e-01, ..., 3.25e+00, 5.60e-01, 9.20e+00]])

In [11]:
xgb_classification = xgb.XGBClassifier()
xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])

[0]	validation_0-merror:0.484375
[1]	validation_0-merror:0.4625
[2]	validation_0-merror:0.45
[3]	validation_0-merror:0.46875
[4]	validation_0-merror:0.46875
[5]	validation_0-merror:0.459375
[6]	validation_0-merror:0.45625
[7]	validation_0-merror:0.453125
[8]	validation_0-merror:0.45
[9]	validation_0-merror:0.459375
[10]	validation_0-merror:0.45
[11]	validation_0-merror:0.45
[12]	validation_0-merror:0.44375
[13]	validation_0-merror:0.45
[14]	validation_0-merror:0.440625
[15]	validation_0-merror:0.44375
[16]	validation_0-merror:0.440625
[17]	validation_0-merror:0.43125
[18]	validation_0-merror:0.428125
[19]	validation_0-merror:0.43125
[20]	validation_0-merror:0.428125
[21]	validation_0-merror:0.428125
[22]	validation_0-merror:0.41875
[23]	validation_0-merror:0.428125
[24]	validation_0-merror:0.421875
[25]	validation_0-merror:0.41875
[26]	validation_0-merror:0.41875
[27]	validation_0-merror:0.425
[28]	validation_0-merror:0.415625
[29]	validation_0-merror:0.4125
[30]	validation_0-merror:0.

XGBClassifier(objective='multi:softprob')

In [12]:
def objective_classification(trial) :
    train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )
    
    param  = {
        'tree_method' :'gpu_hist',
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy

In [13]:
xgb_classification_oputuna = optuna.create_study(direction  = 'minimize')
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[I 2022-09-15 09:16:13,635] A new study created in memory with name: no-name-cdb709d4-046d-4159-9ad6-08e148a908fd


[0]	validation_0-merror:0.540625
[1]	validation_0-merror:0.5375
[2]	validation_0-merror:0.5125
[3]	validation_0-merror:0.521875
[4]	validation_0-merror:0.515625
[5]	validation_0-merror:0.515625
[6]	validation_0-merror:0.509375
[7]	validation_0-merror:0.509375
[8]	validation_0-merror:0.525
[9]	validation_0-merror:0.53125
[10]	validation_0-merror:0.528125
[11]	validation_0-merror:0.525
[12]	validation_0-merror:0.521875
[13]	validation_0-merror:0.521875
[14]	validation_0-merror:0.51875
[15]	validation_0-merror:0.528125
[16]	validation_0-merror:0.525
[17]	validation_0-merror:0.525
[18]	validation_0-merror:0.525
[19]	validation_0-merror:0.525
[20]	validation_0-merror:0.525
[21]	validation_0-merror:0.525
[22]	validation_0-merror:0.525
[23]	validation_0-merror:0.53125
[24]	validation_0-merror:0.534375
[25]	validation_0-merror:0.528125
[26]	validation_0-merror:0.521875
[27]	validation_0-merror:0.521875
[28]	validation_0-merror:0.51875
[29]	validation_0-merror:0.515625
[30]	validation_0-merror:

[I 2022-09-15 09:16:14,116] Trial 0 finished with value: 0.50625 and parameters: {'booster': 'gblinear', 'lambda': 0.437894345899638, 'alpha': 0.060268112408753585, 'subsample': 0.22910887565778368, 'colsample_bytree': 0.4024469677722733}. Best is trial 0 with value: 0.50625.


[09:16:14] GetGradient: 0.0214351s, 100 calls @ 214us
[09:16:14] PredictRaw: 0.00245015s, 100 calls @ 24us
[09:16:14] UpdateOneIter: 0.257639s, 100 calls @ 2576us
[09:16:14] ======== Monitor: GBLinear ========
[09:16:14] DoBoost: 0.233415s, 100 calls @ 2334us
[09:16:14] PredictBatch: 0.0064699s, 202 calls @ 32us
[09:16:14] PredictBatchInternal: 0.1651s, 203 calls @ 813us
[0]	validation_0-merror:0.546875
[1]	validation_0-merror:0.5125
[2]	validation_0-merror:0.525
[3]	validation_0-merror:0.471875
[4]	validation_0-merror:0.453125
[5]	validation_0-merror:0.465625
[6]	validation_0-merror:0.46875
[7]	validation_0-merror:0.471875
[8]	validation_0-merror:0.475
[9]	validation_0-merror:0.465625
[10]	validation_0-merror:0.4625
[11]	validation_0-merror:0.453125
[12]	validation_0-merror:0.465625
[13]	validation_0-merror:0.45
[14]	validation_0-merror:0.45
[15]	validation_0-merror:0.45
[16]	validation_0-merror:0.45
[17]	validation_0-merror:0.44375
[18]	validation_0-merror:0.44375
[19]	validation_0-m

[I 2022-09-15 09:16:16,357] Trial 1 finished with value: 0.615625 and parameters: {'booster': 'gbtree', 'lambda': 0.5760165601233912, 'alpha': 0.5288003427548748, 'subsample': 0.38474696163225985, 'colsample_bytree': 0.29705658951506153}. Best is trial 0 with value: 0.50625.


...

[09:16:29] AllReduce calls: 2853
[09:16:29] AllReduce total MB communicated: 42
[09:16:29] ======== Monitor: HistCutMatrix ========
[09:16:29] ======== Monitor: DeviceShard0 ========
[09:16:29] AllReduce: 0.00798327s, 2253 calls @ 3us
[09:16:29] BuildHist: 0.0457545s, 1653 calls @ 27us
[09:16:29] EvaluateSplits: 0.0720652s, 1653 calls @ 43us
[09:16:29] FinalisePosition: 0.00542892s, 600 calls @ 9us
[09:16:29] InitRoot: 1.67951s, 600 calls @ 2799us
[09:16:29] Reset: 0.0630797s, 600 calls @ 105us
[09:16:29] UpdatePosition: 0.0381587s, 1653 calls @ 23us
[09:16:29] ======== Monitor: HistCutMatrix ========
[0]	validation_0-merror:0.465625
[1]	validation_0-merror:0.478125
[2]	validation_0-merror:0.48125
[3]	validation_0-merror:0.425
[4]	validation_0-merror:0.4375
[5]	validation_0-merror:0.425
[6]	validation_0-merror:0.4375
[7]	validation_0-merror:0.44375
[8]	validation_0-merror:0.425
[9]	validation_0-merror:0.415625
[10]	validation_0-merror:0.41875
[11]	validation_0-merror:0.41875
[12]	valid

[I 2022-09-15 09:16:31,701] Trial 8 finished with value: 0.6375 and parameters: {'booster': 'gbtree', 'lambda': 0.6245214362127814, 'alpha': 0.9324313623796898, 'subsample': 0.4532410129166198, 'colsample_bytree': 0.4138934310835687}. Best is trial 2 with value: 0.48125.


[09:16:31] ======== Monitor: Learner ========
[09:16:31] EvalOneIter: 0.00582605s, 100 calls @ 58us
[09:16:31] GetGradient: 0.00467975s, 100 calls @ 46us
[09:16:31] PredictRaw: 0.0030486s, 100 calls @ 30us
[09:16:31] UpdateOneIter: 1.94914s, 100 calls @ 19491us
[09:16:31] ======== Monitor: GBTree ========
[09:16:31] BoostNewTrees: 1.93313s, 100 calls @ 19331us
[09:16:31] CommitModel: 0.00756568s, 100 calls @ 75us
[09:16:31] ======== Monitor:  ========
[09:16:31] DevicePredictInternal: 0.00784401s, 202 calls @ 38us
[09:16:31] PredictFromCache: 0.00375374s, 199 calls @ 18us
[09:16:31] ======== Monitor: updater_gpu_hist ========
[09:16:31] BinningCompression: 0.000344941s, 1 calls @ 344us
[09:16:31] InitData: 0.016006s, 600 calls @ 26us
[09:16:31] InitDataOnce: 0.0158483s, 1 calls @ 15848us
[09:16:31] Quantiles: 0.00152549s, 1 calls @ 1525us
[09:16:31] Update: 1.90615s, 600 calls @ 3176us
[09:16:31] ======== NCCL Statistics========
[09:16:31] AllReduce calls: 2855
[09:16:31] AllReduce tot

[I 2022-09-15 09:16:35,756] Trial 9 finished with value: 0.6375 and parameters: {'booster': 'dart', 'lambda': 0.034532313597631904, 'alpha': 0.24482842862427118, 'subsample': 0.36792040163214257, 'colsample_bytree': 0.21197696558946852}. Best is trial 2 with value: 0.48125.


[09:16:35] ======== Monitor: Learner ========
[09:16:35] EvalOneIter: 0.384954s, 100 calls @ 3849us
[09:16:35] GetGradient: 0.00727576s, 100 calls @ 72us
[09:16:35] PredictRaw: 1.52526s, 100 calls @ 15252us
[09:16:35] UpdateOneIter: 3.43887s, 100 calls @ 34388us
[09:16:35] ======== Monitor: GBTree ========
[09:16:35] BoostNewTrees: 1.8888s, 100 calls @ 18888us
[09:16:35] CommitModel: 0.0171141s, 100 calls @ 171us
[09:16:35] ======== Monitor:  ========
[09:16:35] ======== Monitor: updater_gpu_hist ========
[09:16:35] BinningCompression: 0.000337006s, 1 calls @ 337us
[09:16:35] InitData: 0.00971319s, 600 calls @ 16us
[09:16:35] InitDataOnce: 0.00959284s, 1 calls @ 9592us
[09:16:35] Quantiles: 0.00157402s, 1 calls @ 1574us
[09:16:35] Update: 1.86435s, 600 calls @ 3107us
[09:16:35] ======== NCCL Statistics========
[09:16:35] AllReduce calls: 2766
[09:16:35] AllReduce total MB communicated: 40
[09:16:35] ======== Monitor: HistCutMatrix ========
[09:16:35] ======== Monitor: DeviceShard0 ====

In [14]:
xgb_classification_oputuna.best_trial

FrozenTrial(number=2, values=[0.48125], datetime_start=datetime.datetime(2022, 9, 15, 9, 16, 16, 358404), datetime_complete=datetime.datetime(2022, 9, 15, 9, 16, 16, 855660), params={'booster': 'gblinear', 'lambda': 0.05388010466089025, 'alpha': 0.16372256075313346, 'subsample': 0.481011972256574, 'colsample_bytree': 0.49432667369065264}, distributions={'booster': CategoricalDistribution(choices=('dart', 'gbtree', 'gblinear')), 'lambda': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'alpha': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'subsample': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'colsample_bytree': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [15]:
param = xgb_classification_oputuna.best_trial.params

In [16]:
xgb_final_clas = xgb.XGBClassifier(**param)
xgb_final_clas.fit(train_x , train_y)

XGBClassifier(alpha=0.16372256075313346, booster='gblinear',
              colsample_bytree=0.49432667369065264, lambda=0.05388010466089025,
              objective='multi:softprob', subsample=0.481011972256574)

In [17]:
xgb_final_clas.score(test_x , test_y)

0.49375